In [1]:
# Installing packages
!pip install qiskit qiskit_nature --quiet
!pip install pyscf qiskit_ibm_runtime --quiet
!pip install prototype-zne --quiet

In [2]:
# Import necessary libraries and packages
import math
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Sampler, Estimator
from qiskit_nature.second_q.circuit.library import UCC, UCCSD, HartreeFock
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper, QubitMapper

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

from qiskit_nature.settings import settings
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit_algorithms.optimizers import SLSQP

from qiskit_algorithms import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.transformers.active_space_transformer import ActiveSpaceTransformer
from qiskit_nature.second_q.problems.electronic_structure_problem import ElectronicStructureProblem
settings.dict_aux_operators = True

In [3]:
# Create an instance of the PySCFDriver with the specified molecular configuration
driver = PySCFDriver(
    atom="Be 0 0 0; H -1.291 0 0; H 1.291 0 0",  # Defines the positions of the atoms
    basis="sto3g",                             # Specifies the basis set for the calculation
    charge=0,                                  # The total charge of the molecule
    spin=0,                                    # The total spin of the molecule
    unit=DistanceUnit.ANGSTROM,                # The unit of measurement for atomic positions
)

# Execute the driver to compute the molecular properties
properties = driver.run()

In [4]:
# Retrieve the number of alpha electrons from the molecular properties
num_alpha_electrons = properties.num_alpha
# Retrieve the number of beta electrons from the molecular properties
num_beta_electrons = properties.num_beta
# Convert the number of spin orbitals to an integer for further calculations
num_spin_orbitals = int(properties.num_spin_orbitals)

# Get the nuclear repulsion energy from the molecular properties
nuclear_rep_energy = properties.nuclear_repulsion_energy

# Print the number of alpha electrons to the console
print("number of alpha electrons: ", num_alpha_electrons)
# Print the number of beta electrons to the console
print("number of beta electrons: ", num_beta_electrons)
# Print the number of spin orbitals to the console
print("number of spin orbitals: ", num_spin_orbitals)

number of alpha electrons:  3
number of beta electrons:  3
number of spin orbitals:  14


In [5]:
# Define the active space around the Fermi level (selected automatically around the HOMO and LUMO, ordered by energy)
transformer = ActiveSpaceTransformer(
    num_electrons=4, #Number of electrons in our active space
    num_spatial_orbitals = 6, #Numer of orbitals in our active space
)

# Define the reduced problem using this transformer
problem = transformer.transform(properties)

# The second quantized Hamiltonian of the reduce problem
second_q_ops_reduced = problem.second_q_ops()

In [6]:
# Setup the mapper and qubit converter
mapper_type = 'ParityMapper'

if mapper_type == 'ParityMapper':
    mapper = ParityMapper()
elif mapper_type == 'JordanWignerMapper':
    mapper = JordanWignerMapper()

In [7]:
# Use Parity mapper to map the problem
qubit_op_parity = mapper.map(problem.hamiltonian.second_q_op())
print(qubit_op_parity)

SparsePauliOp(['IIIIIIIIIIII', 'IIIIIIIIIIIZ', 'IIIIIIIIIIZZ', 'IIIIIIIIIIZI', 'IIIIIIZXXXXZ', 'IIIIIIZXXXXI', 'IIIIIIIYXXYI', 'IIIIIIIYXXYZ', 'IIIIIIIIIZZI', 'IIIIIIIIIZZZ', 'IIIIIIIIZZII', 'IIIIIIIIZZIZ', 'IIIIIIIZZIII', 'IIIIIIIZZIIZ', 'IIIIIIZZIIII', 'IIIIIIZZIIIZ', 'IIIIIZZIIIII', 'IIIIIZZIIIIZ', 'IZXXXXZIIIII', 'IZXXXXZIIIIZ', 'IIYXXYIIIIII', 'IIYXXYIIIIIZ', 'IIIIZZIIIIII', 'IIIIZZIIIIIZ', 'ZXXXXZIIIIII', 'ZXXXXZIIIIIZ', 'IYXXYIIIIIII', 'IYXXYIIIIIIZ', 'IIIZZIIIIIII', 'IIIZZIIIIIIZ', 'IIZZIIIIIIII', 'IIZZIIIIIIIZ', 'IZZIIIIIIIII', 'IZZIIIIIIIIZ', 'ZZIIIIIIIIII', 'ZZIIIIIIIIIZ', 'IIIIIIIZXXXX', 'IIIIIIIZXXYY', 'IIIIIIIIYXXY', 'IIIIIIIIYXYX', 'IIIIIIZXZIZX', 'IIIIIIIXIIZX', 'IIIIIIZYIIIY', 'IIIIIIIYZIZY', 'IIIIIIZXZIIX', 'IIIIIIIXIIIX', 'IIIIZXZIIIZX', 'IIIIIXIIIIZX', 'IIIIZXZIIIIX', 'IIIIIXIIIIIX', 'ZXXXXXZIIIZX', 'IYXXXYIIIIZX', 'ZXXXXXZIIIIX', 'IYXXXYIIIIIX', 'IZXXXZIIIIZX', 'IIYXYIIIIIZX', 'IZXXXZIIIIIX', 'IIYXYIIIIIIX', 'ZXZIIIIIIIZX', 'IXIIIIIIIIZX', 'ZXZIIIIIIIIX', 'IXIIIIII

In [8]:
# Run the sampler job locally using FakeManilaV2
fake_manila = FakeManilaV2()

# Define options for the simulator
options = {"simulator": {"seed_simulator": 42}}

# Create a Sampler object with the fake backend and the defined options
sampler = Sampler(backend=fake_manila, options=options)

# Create an Estimator object with the fake backend and the defined options
estimator = Estimator(backend=fake_manila, options=options)

# Define the ansatz (trial wavefunction) using the UCCSD (Unitary Coupled Cluster Singles and Doubles) method
ansatz = UCCSD(
    problem.num_spatial_orbitals,  # Number of spatial orbitals in the molecule
    problem.num_particles,         # Number of particles (electrons) in the molecule
    mapper,                        # Object that maps fermionic operators to qubit operators
    initial_state=HartreeFock(     # The initial state is set to the Hartree-Fock state
        problem.num_spatial_orbitals,  # Number of spatial orbitals
        problem.num_particles,         # Number of particles (electrons)
        mapper,                        # Mapper for fermionic to qubit operator conversion
    ),
)


In [9]:
# Number of particles after parity reduction (Z_2 symmetry reduction)
problem.num_particles

(2, 2)

In [15]:
# Set up the VQE solver with the estimator, ansatz, and SLSQP optimizer.
vqe_solver = VQE(estimator, ansatz, SLSQP(maxiter=6,ftol = 1e-4))

# Initialize the parameters for the VQE solver to zeros.
vqe_solver.initial_point = [1.0] * ansatz.num_parameters

# Create a ground state eigensolver using the given mapper and VQE solver.
solver = GroundStateEigensolver(mapper, vqe_solver)

# Solve the problem to find the ground state energy of the molecule.
res = solver.solve(problem)

# Output the computed total energy of the molecule's ground state.
print(res.total_energies[0])

AlgorithmError: 'The primitive job to evaluate the energy failed!'

In [16]:
numpy_solver = NumPyMinimumEigensolver()

# Create a ground state eigensolver using the given mapper and Numpy solver.
calc = GroundStateEigensolver(mapper, numpy_solver)
# Solve the problem to find the ground state energy of the molecule.
res = calc.solve(problem)
# Output the computed total energy of the molecule's ground state.
print(res.total_energies[0])


-15.594427466321344


In [13]:
# Using the original problem (without reduction) and Numpy solver
res = calc.solve(properties)
print(res.total_energies[0])

-15.594763661655383


## USING IBM BACKEND

In [ ]:
IBMQ.save_account('')
service = QiskitRuntimeService()

In [ ]:
backend = service.backends(simulator=True)[0]
noisy_sim = FakeGuadalupe()
#for running on real quantum device
#backend = 'ibmq_guadalupe'


print(backend)

In [ ]:
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='qhack-event/main/level-2-team-1',
)

In [ ]:
noise_model = NoiseModel.from_backend(noisy_sim)

options_with_em = Options(
    simulator={
        "noise_model": noise_model,
        "seed_simulator": 42,
    },
    resilience_level=1 # You may change the value here. resilience_level = 1 will activate TREX
)


#for running on real quantum device
#options_with_em = Options(resilience_level=1 # You may change the value here. resilience_level = 1 will activate TREX
#)

In [ ]:
%%time
from qiskit.utils import algorithm_globals
algorithm_globals.random_seed = 1024

# Define convergence list
convergence = []

# Keep track of jobs (Do-not-modify)
job_list = []

# Initialize estimator object
#estimator = Estimator()# Enter your code here

# Define evaluate_expectation function
def evaluate_expectation(x):
    x = list(x)

    # Define estimator run parameters
    #### enter your code below ####



    with Session(service=service, backend=backend):
      estimator = Estimator(options=options_with_em)
      job = estimator.run(circuits=[ansatz], parameter_values=[x],observables=qubit_op_parity).result()
      results = job.values[0]
      job_list.append(job)


    # Pass results back to callback function
    return np.real(results)



# Call back function
def callback(x,fx,ax,tx,nx):
    # Callback function to get a view on internal states and statistics of the optimizer for visualization
    convergence.append(evaluate_expectation(fx))

np.random.seed(10)

# Define initial point. We shall define a random point here based on the number of parameters in our ansatz
initial_point = np.random.random(ansatz.num_parameters)

#### enter your code below ####
# Define optimizer and pass callback function
optimizer = SPSA(maxiter = 50, callback=callback) # ----------- Enter your code here

# Define minimize function
result =  optimizer.minimize(evaluate_expectation, x0=initial_point) # ----------- Enter your code here

In [ ]:
Energy_H_t = []
for i in range(len(convergence)):
    sol = MinimumEigensolverResult()
    sol.eigenvalue = convergence[i]
    sol = problem_reduced.interpret(sol).total_energies[0]
    Energy_H_t.append(sol)
print("Computed Energy:", Energy_H_t[-1])

In [ ]:
# The following plot compares the two Estimators - with and without noise

plt.rcParams["font.size"] = 14

# plot loss and reference value
plt.figure(figsize=(12, 6), facecolor='white')
plt.plot(Energy_H_t, label="Estimator VQE BeH2+NOISY+TREX")
plt.axhline(y=real_solution_t.real, color="tab:red", ls="--", label="Target")

plt.legend(loc="best")
plt.xlabel("Iteration")
plt.ylabel("Energy [H]")
plt.title("VQE energy")
plt.show()

print(Energy_H_t)
print(real_solution_t.real)